In [16]:
import pandas as pd
import numpy as np

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
df = pd.read_csv('/content/drive/MyDrive/Internship Program/songdata.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [19]:
df.shape

(57650, 4)

In [20]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [21]:
df.shape

(5000, 3)

In [22]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [23]:
df['text'][0]

'one baby to another says, "i\'m lucky to have met you"   i don\'t care what you think unless it is about me   it is now my duty to completely drain you   i travel through a tube and end up in your infection   chew my meat for you   pass it back and forth in a passionate kiss   from my mouth to yours   i like you      with eyes so dilated, i\'ve became your pupil   you\'ve taught me everything without a poison apple   the water is so yellow   i\'m a healthy student   indebted and so grateful   vacuum out the fluids   chew my meat for you   pass it back and forth in a passionate kiss   from my mouth to yours   i like you, you, you, you, you, you      one baby to another says, "i\'m lucky to have met you"   i don\'t care what you think unless it is about me   it is now my duty to completely drain you   a travel through a tube and end up in your infection   chew my meat for you   pass it back and forth in a passionate kiss   from my mouth to yours   sloppy lips to lips   you\'re my vitami

In [28]:
import nltk
nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [29]:

df['text'] = df['text'].apply(lambda x: tokenization(x))

In [37]:
df['text']

0       one babi to anoth say , `` i 'm lucki to have ...
1       no more carefre laughter silenc ever after wal...
2       look mama i love you but you got ta let me liv...
3       when i find myself in troubl and i need someon...
4       i ca n't get up anymor , my heart tick so slow...
                              ...                        
4995    tri to be misunderstood but it doe n't do me a...
4996    eh eh eh babi , you 're look fire hot i 'll ha...
4997    you are thousand mile away ooh i miss you darl...
4998    put your sweet lip a littl closer to the phone...
4999    babi smile babi smile babi laugh babi cri do n...
Name: text, Length: 5000, dtype: object

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [32]:
similarity[0]

array([1.        , 0.00827381, 0.00614788, ..., 0.01464899, 0.04712035,
       0.05705569])

In [50]:
df['song'][0]

'Drain You'

In [51]:
df[df['song']=='Knowing Me Knowing You']

,artist,song,text
1,ABBA,Knowing Me Knowing You,no more carefre laughter silenc ever after wal...


In [52]:
# df[df['song']==''].index[0]



# recommedation function

In [48]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [61]:
recommendation('Hate')

["That's A Man's Way",
 'This Man Is Mine',
 'I Wanna Be Your Man',
 'Stand By Your Man',
 "I'm Your Man",
 'Man In Me',
 'I Hate Everyone',
 'No Man Can Love Her Like I Do',
 'Man In Chains',
 'Train Rap With Bonnie Bramlett',
 'Human Spark',
 'Middle Man',
 'I Know You Got A Man',
 'Never Kill Another Man',
 'A New Kind Of Man',
 "Baby, You're A Rich Man",
 'A Part Hate',
 'I Hate Everything',
 'Do Right Woman Do Right Man',
 'Dead Man']

In [54]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))